## Preprocessing

In [71]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [72]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN'])

In [73]:
# Determine the number of unique values in each column.
unique_values_per_column = application_df.nunique()
unique_values_per_column

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [74]:
name_counts = application_df.NAME.value_counts()
name_counts[name_counts < 6]

NAME
FLORIDA FAMILY CHILD CARE HOME ASSOCIATION INC                          5
GERONTOLOGICAL ADVANCED PRACTICE NURSES ASSOCIATION                     5
INTERNATIONL TRANSPLANT NURSES SOCIETY                                  5
NATIONAL ORGANIZATION FOR WOMEN INC                                     5
PTA HAWAII CONGRESS                                                     5
                                                                       ..
ST LOUIS SLAM WOMENS FOOTBALL                                           1
AIESEC ALUMNI IBEROAMERICA CORP                                         1
WEALLBLEEDRED ORG INC                                                   1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI    1
WATERHOUSE CHARITABLE TR                                                1
Name: count, Length: 19214, dtype: int64

In [75]:
for name in name_counts[name_counts < 6].index:
  application_df.NAME = application_df.NAME.replace(name,'Other')

In [76]:
income_amount = application_df.INCOME_AMT.value_counts()
income_amount[income_amount < 100]

Series([], Name: count, dtype: int64)

In [77]:
for income in income_amount[income_amount < 100].index:
  application_df.INCOME_AMT=application_df.INCOME_AMT.replace(income,'Other')

In [78]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print("APPLICATION_TYPE value counts:\n", application_type_counts)


APPLICATION_TYPE value counts:
 APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [79]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff = 200
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()
print("\nCategories to replace with 'Other':", application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()


Categories to replace with 'Other': ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [80]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_type_counts1 = application_df['CLASSIFICATION'].value_counts()
print("CLASSIFICATION value counts:\n", application_type_counts1)


CLASSIFICATION value counts:
 CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [81]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt_1 = application_type_counts1[application_type_counts1 > cutoff]

print("Classifications with counts greater than 1:\n", classification_counts_gt_1)


Classifications with counts greater than 1:
 CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Name: count, dtype: int64


In [82]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
app_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = app_counts[app_counts<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [83]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies=pd.get_dummies(application_df,dtype='int')

In [84]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_df_dummies["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [85]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler= scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [86]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim= len(X_train[0])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                31920     
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 34381 (134.30 KB)
Trainable params: 34381 (134.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [87]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [88]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.4761 - accuracy: 0.7746
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.4348 - accuracy: 0.7935
Epoch 3/50
804/804 [==============================] - 3s 3ms/step - loss: 0.4298 - accuracy: 0.7942
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.4275 - accuracy: 0.7950
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.4255 - accuracy: 0.7959
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.4239 - accuracy: 0.7967
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.4221 - accuracy: 0.7968
Epoch 8/50
804/804 [==============================] - 3s 3ms/step - loss: 0.4218 - accuracy: 0.7988
Epoch 9/50
804/804 [==============================] - 3s 3ms/step - loss: 0.4208 - accuracy: 0.7983
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.4197 - accuracy: 0.7991

In [89]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4549 - accuracy: 0.7900 - 542ms/epoch - 2ms/step
Loss: 0.45489469170570374, Accuracy: 0.789970874786377


In [90]:
# @title Default title text
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.keras')